In [10]:
#Import packages needed
import pandas as pd
import numpy as np
import gmaps
import requests
import re
from config import gkey
from pprint import pprint
gmaps.configure(api_key=gkey)
import matplotlib.pyplot as plt

In [11]:
# Bring in csv and look at data
# jobscat_csv = "../../Archive/completeAndCleanNycJobs_kh.csv"
jobscat_df = pd.read_csv("../../Archive/completeAndCleanNycJobs_kh.csv", dtype={'zip': 'str'})
jobscat_df.shape

(324, 33)

In [13]:
boroughgroup = jobscat_df.groupby(["borough","jobCatCondensed","agency","zip"])\
                                    .agg({'zip': "count", 
                                          "numOfPositions": "sum",
                                         'salaryRangeFrom': "min",
                                         'salaryRangeTo':"max"})
boroughgroup["salaryRangeFrom"] = boroughgroup["salaryRangeFrom"].map("${:,.2f}".format)
boroughgroup["salaryRangeTo"] = boroughgroup["salaryRangeTo"].map("${:,.2f}".format)

boroughgroup = boroughgroup.rename(columns={"borough": "Borough",
                                            "zip":"Jobs in Zip Code",
                                    "numOfPositions":"Number of Positions",
                                    "salaryRangeFrom": "Salary Minimun",
                                    "salaryRangeTo": "Salary Max"})
# df1.rename_axis(['names', None, 'Colors'])
boroughgroup = boroughgroup.rename_axis(["Borough", "Job Category","Department/Agency","Zip Code" ])
boroughgroup

Jobs in Zip Code  \
Borough       Job Category                         Department/Agency              Zip Code                     
Brooklyn      Managerial and Professional          DEPARTMENT OF BUILDINGS        11222.0                  6   
                                                   DEPARTMENT OF TRANSPORTATION   11222.0                  2   
              Operatives and Laborers              DEPARTMENT OF TRANSPORTATION   11222.0                  2   
                                                   DEPT OF ENVIRONMENT PROTECTION 11222.0                  2   
                                                   DEPT. OF HOMELESS SERVICES     11236.0                  4   
              Other                                DEPT OF ENVIRONMENT PROTECTION 10595.0                  2   
                                                                                  11222.0                  1   
                                                   DEPT OF INFO TECH & TELECOMM   11201.0                  4   
                                                   OFFICE OF EMERGENCY MANAGEMENT 11201.0                 12   
                                                   POLICE DEPARTMENT              11216.0                  1   
              Service                              DEPARTMENT OF BUILDINGS        11222.0                  1   
                                                   HRA/DEPT OF SOCIAL SERVICES    11201.0                  1   
                                                   LAW DEPARTMENT                 11201.0                  2   
Manhattan     Managerial and Professional          BOARD OF CORRECTION            10007.0                  2   
                                                   DEPARTMENT OF TRANSPORTATION   10041.0                 16   
                                                   DEPT. OF HOMELESS SERVICES     10004.0                  3   
                                                   HRA/DEPT OF SOCIAL SERVICES    10007.0                  2   
                                                   POLICE DEPARTMENT              10007.0                  1   
              Operatives and Laborers              DEPT OF ENVIRONMENT PROTECTION 10595.0                  2   
                                                   DEPT OF RECORDS & INFO SERVICE 10007.0                  2   
                                                   HRA/DEPT OF SOCIAL SERVICES    10007.0                  1   
              Other                                BOARD OF CORRECTION            10007.0                  4   
                                                   DEPARTMENT OF FINANCE          10007.0                  3   
                                                                                  10038.0                  5   
                                                   DEPARTMENT OF SANITATION       10004.0                  6   
                                                                                  10013.0                  4   
                                                   DEPARTMENT OF TRANSPORTATION   10041.0                 43   
                                                   DEPT OF ENVIRONMENT PROTECTION 10007.0                 10   
                                                                                  10038.0                  4   
                                                                                  12401.0                  2   
                                                                                  12740.0                  2   
                                                   DEPT OF RECORDS & INFO SERVICE 10007.0                  2   
                                                   LAW DEPARTMENT                 10007.0                  4   
                                                   POLICE DEPARTMENT              10007.0                 11   
              Service                              DEPT OF Y

In [15]:
BrooklynGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Brook')])
ManhattanGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Man')])
QueensGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Queens')])
StatenIslandGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('Staten')])
BronxGroup = (boroughgroup[boroughgroup.index.get_level_values("Borough").str.startswith('The Bronx')])

In [16]:
BrooklynGroup.to_html("Brooklyn.html")
ManhattanGroup.to_html("Manhattan.html")
QueensGroup.to_html("Queens.html")
StatenIslandGroup.to_html("StatenIsland.html")
BronxGroup.to_html("Bronx.html")